In [3]:
import torch
torch.set_printoptions(sci_mode=False)
from torch.func import grad, vjp, jacrev
from model_triton import *

aa = torch.randn((2,3)) #torch.randn((2, 2, 3))
print(aa)
print("----")

# RELU
# print(t_relu_bkwd(aa) * aa)
# print(vjp(t_relu_fwd, aa)[0])
#print(jacrev(t_relu_fwd)(aa))

# GELU
# print(jacrev(t_gelu_fwd)(aa))
# print(t_gelu_bkwd(aa))
#print(t_gelu_bkwd(aa) * aa)

# LOG_SOFTMAX
# print(jacrev(t_log_softmax_fwd)(aa))
# print(t_log_softmax_bkwd(aa))

# Embed
# from model_torch_func import init_proj_layer
# params = (init_proj_layer(4, 2), )
# aa =torch.tensor([1, 0, 1])
# print(jacrev(t_embed_fwd)(params, aa))
# print(t_embed_bkwd(params, aa))

# Proj
# aa = torch.randn((2,2))
# #print(aa)
# params = torch.randn((4,2))
# #print(t_proj_fwd(params, aa))
# res = jacrev(t_proj_fwd)(params, aa)
# print(res.shape)
# print(res)
# res2 = t_proj_bkwd_p(params, aa)
# print(res2.shape)
# print(res2)
# res = jacrev(t_proj_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_proj_bkwd_x(params, aa)
# print(res2.shape, res2)

# Linear
# aa = torch.randn((2,2))
# print(aa)
# params = (torch.randn((4,2)), torch.randn((4,)))
# res = jacrev(t_linear_fwd)(params, aa)
# print(res[0].shape, res[1].shape)
# print(res)
# res2 = t_linear_bkwd_p(params, aa)
# print(res2[0].shape, res2[1].shape)
# print(res2)

# from model_triton import t_linear_bkwd_x
# res = jacrev(t_linear_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_linear_bkwd_x(params, aa)
# print(res2.shape, res2)

# LAYERNORM
# x = torch.randn((2,3), device="cuda")
# print(x)
# print(torch.mean(x, axis=-1, keepdims=True), torch.std(x, axis=-1, keepdims = True))
# from model_torch_func import init_layernorm_layer
# #params = init_layernorm_layer(3) # replace 1, 0s
# params = torch.randn((3, ), device="cuda"), torch.zeros((3,), device="cuda")

# from model_triton import t_layernorm_bkwd_p
# res1 = jacrev(t_layernorm_fwd)(params, x)
# res2 = t_layernorm_bkwd_p(params, x)
# print(res1[0].shape, res1[0])
# print("--")
# print(res2[0].shape, res2[0])

# from model_triton import t_layernorm_bkwd_x
# res1 = jacrev(t_layernorm_fwd, argnums=1)(params, x)
# res2 = t_layernorm_bkwd_x(params, x)
# print(res1.shape, res1)
# print("--")
# print(res2.shape, res2)


# TLAYER_FFN
# aa = torch.randn((3,2), device="cuda")
# print(aa)
# from model_torch_func import init_tlayer_ffn
# from model_triton import t_tlayer_ffn_fwd, t_gelu_fwd, t_linear_bkwd_p, t_gelu_bkwd
# from functools import partial
# fn = partial(t_tlayer_ffn_fwd, activation_fn=t_gelu_fwd)
# params = init_tlayer_ffn(2, 4)
# print(fn(params, aa))
# print(params[0].shape, params[1].shape)
# print("--")

# res = jacrev(fn, argnums=0)(params, aa)
# print(res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[0],res[1])
# #print(res[2], res[3])

# res2 = t_tlayer_ffn_bkwd_p(params, aa, t_gelu_fwd)
# print(res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[0], res2[1])
# print(res2[2], res2[3])

# res = jacrev(fn, argnums=1)(params, aa)
# print(res.shape)
# print(res)

# res2 = t_tlayer_ffn_bkwd_x(params, aa, t_gelu_fwd)
# print(res2.shape)
# print(res2)

tensor([[ 1.696425, -0.533204,  0.009236],
        [-1.523902, -1.784682,  1.761061]])
----


In [17]:
# Linear
aa = torch.randn((2,3,2))
print(aa)
params = (torch.randn((4,2)), torch.randn((4,)))
res = jacrev(t_linear_fwd)(params, aa)
print(res[0].shape, res[1].shape)
print(res[1])

def t_linear_bkwd_p(layer_params, x): # input: N x D
    outdim = layer_params[1].shape[0]

    jac1 = t_proj_bkwd_p(layer_params[0], x)
    jac2 = torch.eye(outdim, device=x.device).expand(x.shape[:-1] + (outdim, outdim))
    return jac1, jac2

res2 = t_linear_bkwd_p(params, aa)
print(res2[0].shape, res2[1].shape)
print(res2[1])

tensor([[[-1.764597,  0.521342],
         [ 1.225475,  1.405953],
         [-0.352109,  1.071642]],

        [[-0.258679, -0.220236],
         [ 0.091375, -0.252992],
         [-0.352096, -0.383530]]])
torch.Size([2, 3, 4, 4, 2]) torch.Size([2, 3, 4, 4])
tensor([[[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]],


        [[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]],

         [[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]]])
torch.Size([2, 3, 4, 4, 2]) torch.Size([2, 3, 4, 4])
tenso

In [3]:
aa = torch.randn((1, 3,2), device="cuda")
aa

tensor([[[-0.901137,  1.467101],
         [-0.212698,  0.167734],
         [-1.033636,  1.201384]]], device='cuda:0')

In [4]:
# dropout train=False (Note that this jacobian is not needed..)
from torch.func import grad, vjp, jacrev
from model_triton import *
res = t_dropout_fwd(aa, train=False)
from functools import partial
print(res)

fn = partial(t_dropout_fwd, train=False)
res = jacrev(fn)(aa)
print(res.shape, res)

res2 = t_dropout_bkwd(aa, train=False)
print(res2.shape, res2)

tensor([[[-0.811023,  1.320391],
         [-0.191428,  0.150961],
         [-0.930272,  1.081246]]], device='cuda:0')
torch.Size([1, 3, 2, 1, 3, 2]) tensor([[[[[[0.900000, 0.000000],
            [0.000000, 0.000000],
            [0.000000, 0.000000]]],


          [[[0.000000, 0.900000],
            [0.000000, 0.000000],
            [0.000000, 0.000000]]]],



         [[[[0.000000, 0.000000],
            [0.900000, 0.000000],
            [0.000000, 0.000000]]],


          [[[0.000000, 0.000000],
            [0.000000, 0.900000],
            [0.000000, 0.000000]]]],



         [[[[0.000000, 0.000000],
            [0.000000, 0.000000],
            [0.900000, 0.000000]]],


          [[[0.000000, 0.000000],
            [0.000000, 0.000000],
            [0.000000, 0.900000]]]]]], device='cuda:0')
torch.Size([1, 3, 2, 1, 3, 2]) tensor([[[[[[0.900000, 0.000000],
            [0.000000, 0.000000],
            [0.000000, 0.000000]]],


          [[[0.000000, 0.900000],
            [0.000000,

In [5]:
# dropout train=True
from torch.func import grad, vjp, jacrev
from model_triton import *
p_gen_aux = 42

res = t_dropout_fwd(aa, train=True, p_gen_aux=p_gen_aux)
from functools import partial
#print(res)

fn = partial(t_dropout_fwd, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(aa)
print(res.shape, res)
print(f'---')

res2 = t_dropout_bkwd(aa, train=True, p_gen_aux=p_gen_aux)
print(res2.shape, res2)

torch.Size([1, 3, 2, 1, 3, 2]) tensor([[[[[[1., 0.],
            [0., 0.],
            [0., 0.]]],


          [[[0., 1.],
            [0., 0.],
            [0., 0.]]]],



         [[[[0., 0.],
            [1., 0.],
            [0., 0.]]],


          [[[0., 0.],
            [0., 1.],
            [0., 0.]]]],



         [[[[0., 0.],
            [0., 0.],
            [1., 0.]]],


          [[[0., 0.],
            [0., 0.],
            [0., 1.]]]]]], device='cuda:0')
---
torch.Size([1, 3, 2, 1, 3, 2]) tensor([[[[[[1., 0.],
            [0., 0.],
            [0., 0.]]],


          [[[0., 1.],
            [0., 0.],
            [0., 0.]]]],



         [[[[0., 0.],
            [1., 0.],
            [0., 0.]]],


          [[[0., 0.],
            [0., 1.],
            [0., 0.]]]],



         [[[[0., 0.],
            [0., 0.],
            [1., 0.]]],


          [[[0., 0.],
            [0., 0.],
            [0., 1.]]]]]], device='cuda:0')


In [40]:
# Embed
from model_torch_func import init_proj_layer
params = (init_proj_layer(4, 2), )
#aa =torch.tensor([1, 0, 1])
aa = torch.randint(2, (2, 6))
res=jacrev(t_embed_fwd)(params, aa)
print(len(res), res[0].shape)
print(res[0])

res2=t_embed_bkwd(params, aa)
print(len(res2), res2[0].shape)
print(res2[0])

1 torch.Size([2, 6, 4, 2, 4])
tensor([[[[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


         [[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


         [[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


         [[[2., 0., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 2., 0., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 2., 0.],
           [0., 0., 0., 0.]],

          [[0., 0., 0., 2.],
           [0., 0., 0., 0.]]],


  

In [2]:
# TLAYER_FFN
aa = torch.randn((1, 3,2), device="cuda")
print(aa)
from model_torch_func import init_tlayer_ffn
from model_triton import t_tlayer_ffn_fwd, t_gelu_fwd, t_linear_bkwd_p, t_gelu_bkwd
from functools import partial
fn = partial(t_tlayer_ffn_fwd, activation_fn=t_gelu_fwd)
params = init_tlayer_ffn(2, 4)
print(fn(params, aa))
print(params[0].shape, params[1].shape)
print("--")

# res = jacrev(fn, argnums=0)(params, aa)
# print(res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[0],res[1])
# print(res[2], res[3])
# print("--")

# res2 = t_tlayer_ffn_bkwd_p(params, aa, t_gelu_fwd)
# print(res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[0], res2[1])
# print(res2[2], res2[3])

res = jacrev(fn, argnums=1)(params, aa)
print(res.shape)
print(res)
print("--")

# def t_tlayer_ffn_bkwd_x(layer_params, x, activation_fn):
#     x_2d = x.reshape((-1, x.shape[-1]))
    
#     act_fn_bkwd = t_gelu_bkwd if activation_fn==t_gelu_fwd else t_relu_bkwd
    
#     dffn1_dx = t_linear_bkwd_x((layer_params[0], layer_params[1]), x_2d)
#     x_2d = t_linear_fwd((layer_params[0], layer_params[1]), x_2d)
#     dact_dx = act_fn_bkwd(x_2d)
#     x_2d = activation_fn(x_2d)
#     dffn2_dx = t_linear_bkwd_x((layer_params[2], layer_params[3]), x_2d)
#     dffn2_act_dx = dact_dx * dffn2_dx #Note dact_dx is only 2D, but torch will add other dims
#     jac = torch.einsum('abcd,cdef->abef', dffn2_act_dx, dffn1_dx)
#     return jac.reshape(x.shape+x.shape)

res2 = t_tlayer_ffn_bkwd_x(params, aa, t_gelu_fwd)
print(res2.shape)
print(res2)

tensor([[[-0.9504,  0.5064],
         [-1.1646, -0.3184],
         [ 0.3456,  0.1814]]], device='cuda:0')
tensor([[[    -0.0001,     -0.0001],
         [     0.0002,      0.0002],
         [    -0.0001,     -0.0001]]], device='cuda:0')
torch.Size([4, 2]) torch.Size([4])
--
torch.Size([1, 3, 2, 1, 3, 2])
tensor([[[[[[    -0.0001,     -0.0004],
            [     0.0000,      0.0000],
            [     0.0000,      0.0000]]],


          [[[    -0.0001,     -0.0003],
            [     0.0000,      0.0000],
            [     0.0000,      0.0000]]]],



         [[[[     0.0000,      0.0000],
            [    -0.0001,     -0.0004],
            [     0.0000,      0.0000]]],


          [[[     0.0000,      0.0000],
            [    -0.0001,     -0.0003],
            [     0.0000,      0.0000]]]],



         [[[[     0.0000,      0.0000],
            [     0.0000,      0.0000],
            [    -0.0001,     -0.0004]]],


          [[[     0.0000,      0.0000],
            [     0.0000,      

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:767: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [2]:
import torch
from model_triton import *
aa = torch.arange(8).reshape((2,2,2))
params = torch.arange(12).reshape((2, 3, 2))

print(torch.matmul(aa, params.reshape((-1, params.shape[-1])).transpose(-2, -1)).)
print(torch.matmul(aa, params.transpose(-2, -1)))

SyntaxError: invalid syntax (454213176.py, line 6)

In [1]:
# Proj
import torch
torch.set_printoptions(sci_mode=False)
from torch.func import grad, vjp, jacrev
from model_triton import *
aa = torch.randn((1, 3, 2, 2))
#print(aa)
params = torch.randn((2, 3, 3, 2))
output = t_proj_fwd(params, aa)
print(output.shape, output)
print("--")
res = jacrev(t_proj_fwd)(params, aa)
print(res.shape)
#print(res)
print("--")
res2 = t_proj_bkwd_p(params, aa)
print(res2.shape)
#print(res2)
# res = jacrev(t_proj_fwd, argnums=1)(params, aa)
# print(res.shape, res)
# res2 = t_proj_bkwd_x(params, aa)
# print(res2.shape, res2)

torch.Size([2, 3, 2, 3]) tensor([[[[ 0.7874, -0.8682,  1.7077],
          [ 0.3367,  2.4793, -0.9132]],

         [[-0.6759,  0.5020,  2.1995],
          [-0.4050,  0.4044, -1.4075]],

         [[ 1.8927,  0.2770,  0.1400],
          [-2.3340, -0.5911,  1.1458]]],


        [[[ 2.0719,  0.8006, -0.3591],
          [ 0.8510, -2.5076, -1.9545]],

         [[-0.3017,  0.7745, -2.8226],
          [ 0.2864, -0.8024,  1.7334]],

         [[ 1.6732, -1.1712,  0.1737],
          [ 1.7747, -0.5364, -1.2671]]]])
--
torch.Size([2, 3, 2, 3, 2, 3, 3, 2])
--
torch.Size([2, 3, 2, 3, 2, 3, 3, 2])


In [1]:
# t_scaled_dot_prod_attn: numerical instablity in attn_bkwd
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 4 #Works for 3, 2, 2, 1!
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

def attn_fwd(q, k, mask):
    attn = torch.matmul(q, torch.transpose(k, -2, -1))
    attn = attn / math.sqrt(D)
    attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
    return torch.exp(t_log_softmax_fwd(attn)) #TODO XXX: numerical stability

q, k, _ = torch.unbind(qkv, dim=2)
# #fn = partial(attn_fwd, mask=mask) #this works
fn = partial(t_softmax_attn_fwd, mask=mask, train=False)
res1 = jacrev(fn)(q, k)
res2 = jacrev(fn, argnums=1)(q, k)
print(res1.shape, res2.shape)
print(res1)
#print(res2)
print("--")

### Repro for numerical isntability!
def attn_bkwd(q, k, mask): # Do all ops in 3d instead of 4d
    attn = torch.matmul(q, torch.transpose(k, -2, -1))
    attn = attn / math.sqrt(D)
    attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
    dattn_dx = torch.exp(t_log_softmax_fwd(attn))[..., None, None, None, None] * t_log_softmax_bkwd(attn) 
    #dattn_dx = t_log_softmax_bkwd(attn) # TODO XXX: numerical stability
    jac1 = torch.matmul(dattn_dx, k/math.sqrt(D))
    jac2 = torch.matmul(q.transpose(-2,-1)/math.sqrt(D), dattn_dx).transpose(-2,-1)
    return jac1, jac2

# This works modulo numerical instability
# res2 = attn_bkwd(q, k, mask)
# print(res2[0].shape, res2[1].shape)
# print(res2[0])
# print(res2[1])

from model_triton import _mult_jacs_in_2d

# def t_softmax_attn_bkwd(q, k, mask, train):
#     D = q.shape[-1]
#     attn = torch.matmul(q, torch.transpose(k, -2, -1))
#     attn = attn / math.sqrt(D)
#     attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
#     # TODO XXX: would the below line cause numerical stabliity issues?
#     sa = torch.exp(t_log_softmax_fwd(attn)) 
#     #sa = t_dropout_fwd(sa, train)
#     jac_dropout = t_dropout_bkwd(sa, train)
#     #TODO: Note, we are overloading _mult.., as right is not Jacobian...
#     sa = _mult_jacs_in_2d(jac_dropout, [sa], sa)[0] 
    
#     # TODO XXX: Clean up below..
#     d_dropout_dx = 1 #0.9 # TODO: XXX add proper dropout 
#     jac_sa_x = d_dropout_dx * sa[..., None, None, None, None] * t_log_softmax_bkwd(attn)
#     jac1 = torch.matmul(jac_sa_x, k/math.sqrt(D))
#     jac2 = torch.matmul(q.transpose(-2,-1), jac_sa_x/math.sqrt(D)).transpose(-2,-1)
#     # Account for mask:
#     jac_mask = torch.unsqueeze(mask,dim=1)[..., None, None, None, None]
#     jac1 = torch.where(jac_mask, jac1, 0)
#     jac2 = torch.where(jac_mask, jac2, 0)
#     return jac1, jac2

# This works modulo numerical instability
res2 = t_softmax_attn_bkwd(q, k, mask, False)
print(res2[0].shape, res2[1].shape)
print(res2[0])
#print(res2[1])

torch.Size([1, 1, 2, 2, 1, 1, 2, 4]) torch.Size([1, 1, 2, 2, 1, 1, 2, 4])
tensor([[[[[[[[ 0.001104, -0.000665, -0.001077,  0.000701],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[-0.001104,  0.000665,  0.001077, -0.000701],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [ 0.001111, -0.000669, -0.001084,  0.000706]]]],



           [[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [-0.001111,  0.000669,  0.001084, -0.000706]]]]]]]],
       device='cuda:0')
--
torch.Size([1, 1, 2, 2, 1, 1, 2, 4]) torch.Size([1, 1, 2, 2, 1, 1, 2, 4])
tensor([[[[[[[[ 0.001104, -0.000665, -0.001077,  0.000701],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[-0.001104,  0.000665,  0.001077, -0.000701],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000,  0.000000,  0.000000],
    

In [2]:
# t_softmax_attn_bkwd: mask + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 2 #Works for 3, 2, 2, 1!
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
# for i, i_mask in enumerate(mask):
#     mask[i] = torch.tril(i_mask)
#    mask[i] = torch.zeros_like(i_mask)
from functools import partial

q, k, _ = torch.unbind(qkv, dim=2)
# #fn = partial(attn_fwd, mask=mask) #this works
fn = partial(t_softmax_attn_fwd, mask=mask, train=False)
res1 = jacrev(fn)(q, k)
res2 = jacrev(fn, argnums=1)(q, k)
#print(res1.shape, res2.shape)
print(res1)
print(res2)
print("--")

# This works modulo numerical instability
res2 = t_softmax_attn_bkwd(q, k, mask, False)
print(res2[0].shape, res2[1].shape)
print(res2[0])
print(res2[1])

tensor([[[[[[[[-0.011142,  0.017495],
              [ 0.000000,  0.000000]]]],



           [[[[ 0.011142, -0.017495],
              [ 0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000],
              [-0.143701,  0.225636]]]],



           [[[[ 0.000000,  0.000000],
              [ 0.143701, -0.225636]]]]]]]], device='cuda:0')
tensor([[[[[[[[-0.013766,  0.014875],
              [ 0.013766, -0.014875]]]],



           [[[[ 0.013766, -0.014875],
              [-0.013766,  0.014875]]]]],




          [[[[[-0.106701,  0.024888],
              [ 0.106701, -0.024888]]]],



           [[[[ 0.106701, -0.024888],
              [-0.106701,  0.024888]]]]]]]], device='cuda:0')
--
torch.Size([1, 1, 2, 2, 1, 1, 2, 2]) torch.Size([1, 1, 2, 2, 1, 1, 2, 2])
tensor([[[[[[[[-0.011142,  0.017495],
              [ 0.000000,  0.000000]]]],



           [[[[ 0.011142, -0.017495],
              [ 0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000],
              [-0.14370

In [2]:
# t_softmax_attn_bkwd: mask + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 2 #Works for 3, 2, 2, 1!
p_gen_aux = 42
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
# for i, i_mask in enumerate(mask):
#     mask[i] = torch.tril(i_mask)
#    mask[i] = torch.zeros_like(i_mask)
from functools import partial

q, k, _ = torch.unbind(qkv, dim=2)
# #fn = partial(attn_fwd, mask=mask) #this works
fn = partial(t_softmax_attn_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
res1 = jacrev(fn)(q, k)
res2 = jacrev(fn, argnums=1)(q, k)
#print(res1.shape, res2.shape)
print(res1)
print(res2)
print("--")

# This works modulo numerical instability
res2 = t_softmax_attn_bkwd(q, k, mask, True, p_gen_aux)
print(res2[0].shape, res2[1].shape)
print(res2[0])
print(res2[1])

tensor([[[[[[[[ 0.018750, -0.003784],
              [ 0.000000,  0.000000]]]],



           [[[[-0.018750,  0.003784],
              [ 0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000],
              [ 0.018709, -0.003776]]]],



           [[[[ 0.000000,  0.000000],
              [-0.018709,  0.003776]]]]]]]], device='cuda:0')
tensor([[[[[[[[-0.106762, -0.111337],
              [ 0.106762,  0.111337]]]],



           [[[[ 0.106762,  0.111337],
              [-0.106762, -0.111337]]]]],




          [[[[[-0.226691,  0.041643],
              [ 0.226691, -0.041643]]]],



           [[[[ 0.226691, -0.041643],
              [-0.226691,  0.041643]]]]]]]], device='cuda:0')
--
torch.Size([1, 1, 2, 2, 1, 1, 2, 2]) torch.Size([1, 1, 2, 2, 1, 1, 2, 2])
tensor([[[[[[[[ 0.018750, -0.003784],
              [ 0.000000,  0.000000]]]],



           [[[[-0.018750,  0.003784],
              [ 0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000],
              [ 0.01870

In [3]:
# t_scaled_dot_prod_attn + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 4
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

#output1 = t_scaled_dot_prod_attn_fwd(qkv, mask, False)

fn = partial(t_scaled_dot_prod_attn_fwd, mask=mask, train=False)
res = jacrev(fn)(qkv)
# print(res.shape)
dq,dk,dv = res.unbind(-3)
print(dq.shape, dk.shape, dv.shape)
print(dq)
#print(dk)
#print(dv)
print("--")

from model_triton import _mult_jacs_in_2d

res2 = t_scaled_dot_prod_attn_bkwd(qkv, mask, False)
print(res2[0].shape, res2[1].shape, res2[2].shape)
print(res2[0])
#print(res2[1])
#print(res2[2])

torch.Size([1, 1, 2, 4, 1, 1, 2, 4]) torch.Size([1, 1, 2, 4, 1, 1, 2, 4]) torch.Size([1, 1, 2, 4, 1, 1, 2, 4])
tensor([[[[[[[[-0.036859,  0.082568,  0.102044, -0.241970],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[ 0.012432, -0.027849, -0.034417,  0.081612],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[ 0.045745, -0.102473, -0.126645,  0.300305],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[ 0.005014, -0.011233, -0.013882,  0.032919],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [-0.035206,  0.078865,  0.097468, -0.231120]]]],



           [[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [ 0.011874, -0.026600, -0.032874,  0.077953]]]],



           [[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [ 0.043694, -0.097878, -0.120966,  0.286840]]]],



     

In [1]:
# t_scaled_dot_prod_attn + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_triton import *
BS, H, N, D = 1, 1, 2, 4
p_gen_aux = 42
qkv= torch.randn((BS, H, 3, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

#output1 = t_scaled_dot_prod_attn_fwd(qkv, mask, False)

fn = partial(t_scaled_dot_prod_attn_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(qkv)
# print(res.shape)
dq,dk,dv = res.unbind(-3)
print(dq.shape, dk.shape, dv.shape)
print(dq)
#print(dk)
#print(dv)
print("--")

from model_triton import _mult_jacs_in_2d

res2 = t_scaled_dot_prod_attn_bkwd(qkv, mask, True, p_gen_aux)
print(res2[0].shape, res2[1].shape, res2[2].shape)
print(res2[0])
#print(res2[1])
#print(res2[2])

torch.Size([1, 1, 2, 4, 1, 1, 2, 4]) torch.Size([1, 1, 2, 4, 1, 1, 2, 4]) torch.Size([1, 1, 2, 4, 1, 1, 2, 4])
tensor([[[[[[[[-0.041449,  0.025426, -0.049119,  0.098434],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[ 0.040940, -0.025114,  0.048515, -0.097226],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[-0.127264,  0.078069, -0.150814,  0.302233],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]],



           [[[[-0.026101,  0.016011, -0.030930,  0.061985],
              [ 0.000000,  0.000000,  0.000000,  0.000000]]]]],




          [[[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [-0.035494,  0.021774, -0.042063,  0.084294]]]],



           [[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [ 0.035059, -0.021506,  0.041546, -0.083259]]]],



           [[[[ 0.000000,  0.000000,  0.000000,  0.000000],
              [-0.108982,  0.066854, -0.129149,  0.258816]]]],



     

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:767: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [2]:
# t_tlayer_attn_heads_fwd + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn_heads
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
layer_params = init_tlayer_attn_heads(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

output = t_tlayer_attn_heads_fwd(layer_params, qkv, mask=mask, train=False)
print("layer_params", layer_params.shape)
print("output", output.shape)

# fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=False)
# res = jacrev(fn)(layer_params, qkv)
# print(res.shape)
# print(res)
# print("--")

from model_triton import _mult_jacs_in_2d

# print("--")
# res2 = t_tlayer_attn_heads_bkwd_p(layer_params, qkv, mask, False)
# print(res2.shape)
# print(res)

fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=False)
res = jacrev(fn, argnums=1)(layer_params, qkv)
print(len(res), res[0].shape, res[1].shape, res[2].shape)
print(res[0])
print("--")

res2 = t_tlayer_attn_heads_bkwd_x(layer_params, qkv, mask, False)
#print(res2.shape)
print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape)
print(res2[0])

layer_params torch.Size([2, 3, 2, 4])
output torch.Size([1, 2, 2, 2])
3 torch.Size([1, 2, 2, 2, 1, 2, 4]) torch.Size([1, 2, 2, 2, 1, 2, 4]) torch.Size([1, 2, 2, 2, 1, 2, 4])
tensor([[[[[[[-9.023733e-06, -5.758845e-06, -1.051356e-06,  5.490635e-06],
             [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


           [[[-5.143159e-06, -3.282307e-06, -5.992299e-07,  3.129438e-06],
             [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]]],



          [[[[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
             [-9.023734e-06, -5.758847e-06, -1.051356e-06,  5.490637e-06]]],


           [[[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
             [-5.143162e-06, -3.282309e-06, -5.992304e-07,  3.129440e-06]]]]],




         [[[[[-3.768324e-06, -1.482875e-06,  9.540058e-07,  2.383768e-06],
             [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


           [[[ 5.717741e-06,  2.249991e-06, -1.447

In [2]:
# t_tlayer_attn_heads_fwd + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn_heads
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
p_gen_aux = 42
layer_params = init_tlayer_attn_heads(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial

output = t_tlayer_attn_heads_fwd(layer_params, qkv, mask=mask, train=True, p_gen_aux=p_gen_aux)
print("layer_params", layer_params.shape)
print("output", output.shape)

fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=False)
res = jacrev(fn)(layer_params, qkv)
print(res.shape)
print(res)
print("--")

res2 = t_tlayer_attn_heads_bkwd_p(layer_params, qkv, mask, False)
print(res2.shape)
print(res)

# fn = partial(t_tlayer_attn_heads_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
# res = jacrev(fn, argnums=1)(layer_params, qkv)
# print(len(res), res[0].shape, res[1].shape, res[2].shape)
# print(res[0])
# print("--")

# res2 = t_tlayer_attn_heads_bkwd_x(layer_params, qkv, mask, True, p_gen_aux)
# #print(res2.shape)
# print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape)
# print(res2[0])

layer_params torch.Size([2, 3, 2, 4])
output torch.Size([1, 2, 2, 2])
torch.Size([1, 2, 2, 2, 2, 3, 2, 4])
tensor([[[[[[[[ 5.740308e-07, -1.442009e-06,  3.217673e-06, -1.204639e-05],
              [ 2.437348e-06, -6.122803e-06,  1.366232e-05, -5.114925e-05]],

             [[ 3.343014e-05, -4.790054e-06,  3.907021e-05,  3.111871e-05],
              [ 2.655931e-05, -3.805566e-06,  3.104018e-05,  2.472294e-05]],

             [[-7.261819e-01, -8.083431e-01, -5.263261e-01, -3.129264e-01],
              [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]]],


            [[[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
              [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]],

             [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
              [ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00]],

             [[ 0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00],
              [ 0.000000e+00,  0.000000e+00, 

In [4]:
# t_tlayer_attn_fwd + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn
from model_triton import *
BS, H, N, D = 1, 2, 2, 4 # It doesn't work for 1, 2, 2, 2
layer_params = init_tlayer_attn(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial
fn = partial(t_tlayer_attn_fwd, mask=mask, train=False)
res = jacrev(fn)(layer_params, qkv)
print(len(res), res[0].shape, res[1].shape)
print(res[0][0][0])
print(res[1][0][0])
#print(res)
print("--")

res2 = t_tlayer_attn_bkwd_p(layer_params, qkv, mask, False)
print("--")
print(len(res2), res2[0].shape, res2[1].shape)
print(res2[0][0][0])
print(res2[1][0][0])

# ####

# res = jacrev(fn, argnums=1)(layer_params, qkv)
# print(type(res), len(res), res[0].shape, res[1].shape)
# print(res[0][0])
# print(f'--')

# from model_triton import _mult_jacs_in_2d

# res2 = t_tlayer_attn_bkwd_x(layer_params, qkv, mask, False)
# print(f'--')
# print(type(res2), len(res2), res2[0].shape, res2[1].shape)
# print(res2[0][0])

2 torch.Size([1, 2, 4, 2, 3, 2, 4]) torch.Size([1, 2, 4, 4, 4])
tensor([[[[[ 6.278919e-07,  1.590083e-06, -1.384409e-06, -7.190078e-07],
           [-2.743195e-06, -6.946909e-06,  6.048343e-06,  3.141271e-06]],

          [[ 2.412633e-06,  1.746262e-06, -5.243527e-06,  1.698564e-06],
           [-7.732598e-07, -5.596850e-07,  1.680574e-06, -5.443975e-07]],

          [[ 5.128642e-03,  2.609785e-03,  1.131797e-02,  2.197274e-02],
           [ 1.001887e-03,  5.098252e-04,  2.210982e-03,  4.292406e-03]]],


         [[[-9.239161e-08, -2.339739e-07,  2.037099e-07,  1.057989e-07],
           [ 7.573799e-08,  1.918000e-07, -1.669911e-07, -8.672863e-08]],

          [[ 8.058645e-08,  5.832843e-08, -1.751436e-07,  5.673522e-08],
           [-6.968085e-08, -5.043496e-08,  1.514418e-07, -4.905736e-08]],

          [[-2.393427e-03, -1.218390e-03, -5.280560e-03, -1.024892e-02],
           [-1.412790e-02, -7.191901e-03, -3.117005e-02, -6.049726e-02]]]],



        [[[[ 7.319467e-08,  1.853593e-07, 

In [2]:
# t_tlayer_attn_fwd + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_attn
from model_triton import *
BS, H, N, D = 1, 2, 2, 4 # It doesn't work for 1, 2, 2, 2
p_gen_aux = 42
layer_params = init_tlayer_attn(D, H)
qkv= torch.randn((BS, 3, N, D), device="cuda").unbind(-3)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
from functools import partial
fn = partial(t_tlayer_attn_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layer_params, qkv)
print(len(res), res[0].shape, res[1].shape)
print(res[0][0][0])
print(res[1][0][0])
#print(res)
print("--")

res2 = t_tlayer_attn_bkwd_p(layer_params, qkv, mask, True, p_gen_aux)
print("--")
print(len(res2), res2[0].shape, res2[1].shape)
print(res2[0][0][0])
print(res2[1][0][0])

# ####

# res = jacrev(fn, argnums=1)(layer_params, qkv)
# print(type(res), len(res), res[0].shape, res[1].shape)
# print(res[0][0])
# print(f'--')

# from model_triton import _mult_jacs_in_2d

# res2 = t_tlayer_attn_bkwd_x(layer_params, qkv, mask, False)
# print(f'--')
# print(type(res2), len(res2), res2[0].shape, res2[1].shape)
# print(res2[0][0])

2 torch.Size([1, 2, 4, 2, 3, 2, 4]) torch.Size([1, 2, 4, 4, 4])
tensor([[[[[ 9.888595e-06, -1.494707e-05,  9.430310e-06, -1.461163e-05],
           [-1.096886e-06,  1.657994e-06, -1.046051e-06,  1.620786e-06]],

          [[ 1.617867e-05, -2.441726e-05, -3.441375e-06,  1.620722e-05],
           [ 1.399617e-05, -2.112338e-05, -2.977134e-06,  1.402087e-05]],

          [[-1.827737e-02, -3.571232e-02,  3.943164e-02,  8.377606e-03],
           [-1.575727e-02, -3.078827e-02,  3.399476e-02,  7.222490e-03]]],


         [[[-4.840731e-06,  7.316990e-06, -4.616389e-06,  7.152784e-06],
           [-6.914753e-07,  1.045197e-06, -6.594290e-07,  1.021741e-06]],

          [[ 2.600796e-06, -3.925188e-06, -5.532168e-07,  2.605384e-06],
           [ 6.263246e-06, -9.452651e-06, -1.332259e-06,  6.274297e-06]],

          [[ 1.305933e-02,  2.551670e-02, -2.815323e-02, -6.026927e-03],
           [ 5.355588e-03,  1.046432e-02, -1.154555e-02, -2.471623e-03]]]],



        [[[[ 2.767318e-06, -4.182930e-06, 

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:767: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /opt/pytorch/pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [6]:
# t_gpt2_tlayer_fwd_sublock1_fwd Train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, False)
print(res.shape)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock1_fwd, mask=mask, train=False)
res = jacrev(fn)(layer_params[:-6], y)
# print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[1])
# #print(res[1])
# print("---")

from model_triton import _mult_jacs_in_2d

# res2 = t_gpt2_tlayer_sublock1_bkwd_p(layer_params[:-6], y, mask, train=False)
# print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[1])

res = jacrev(fn, argnums=1)(layer_params[:-6], y)
print(res.shape, res)
print("---")

res2 = t_gpt2_tlayer_sublock1_bkwd_x(layer_params[:-6], y, mask, train=False)
print(res2.shape, res2)

torch.Size([1, 2, 4])
---
torch.Size([1, 2, 4, 1, 2, 4]) tensor([[[[[[ 9.996231e-01,  1.438149e-04,  5.101071e-05,  1.821585e-04],
            [-4.071151e-04,  2.044572e-04, -2.376328e-05,  2.264212e-04]]],


          [[[-1.047538e-04,  1.000091e+00, -2.112311e-05,  3.476722e-05],
            [-1.282808e-04,  2.834288e-05, -3.153478e-05,  1.314727e-04]]],


          [[[ 8.463881e-05, -4.487316e-05,  9.999972e-01, -3.699768e-05],
            [ 1.001096e-04, -1.238897e-04, -4.321824e-05,  6.699841e-05]]],


          [[[ 2.538253e-05, -6.186862e-05,  3.257946e-05,  1.000004e+00],
            [ 5.246916e-05, -1.176246e-04, -5.776919e-05,  1.229246e-04]]]],



         [[[[-3.770274e-04,  1.437872e-04,  5.104710e-05,  1.821931e-04],
            [ 9.995930e-01,  2.044537e-04, -2.375363e-05,  2.263845e-04]]],


          [[[-1.046268e-04,  9.099802e-05, -2.109664e-05,  3.472545e-05],
            [-1.284503e-04,  1.000028e+00, -3.145642e-05,  1.313463e-04]]],


          [[[ 8.426186e-05, -

In [5]:
# t_gpt2_tlayer_fwd_sublock1_fwd Train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
p_gen_aux = [42, 43]
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, True, p_gen_aux)
print(res.shape)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock1_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layer_params[:-6], y)
print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
print(res[1])
#print(res[1])
print("---")

res2 = t_gpt2_tlayer_sublock1_bkwd_p(layer_params[:-6], y, mask, train=True, p_gen_aux=p_gen_aux)
print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
print(res2[1])

# res = jacrev(fn, argnums=1)(layer_params[:-6], y)
# print(res.shape, res)
# print("---")

# res2 = t_gpt2_tlayer_sublock1_bkwd_x(layer_params[:-6], y, mask, train=True, p_gen_aux=p_gen_aux)
# print(res2.shape, res2)

torch.Size([1, 2, 4])
---
4 torch.Size([1, 2, 4, 4]) torch.Size([1, 2, 4, 4]) torch.Size([1, 2, 4, 2, 3, 2, 4]) torch.Size([1, 2, 4, 4, 4])
tensor([[[[-1.228441e-04,  1.492486e-04, -4.761901e-04, -1.169280e-04],
          [-5.437012e-04,  2.777719e-04,  1.680864e-04, -7.518192e-06],
          [ 6.221797e-05, -4.882399e-04, -1.135939e-04,  9.591794e-04],
          [-5.936526e-04,  4.759873e-04, -2.091079e-04, -3.034194e-04]],

         [[-1.228441e-04,  1.492486e-04, -4.761901e-04, -1.169280e-04],
          [-5.437012e-04,  2.777719e-04,  1.680864e-04, -7.518185e-06],
          [ 6.221791e-05, -4.882399e-04, -1.135939e-04,  9.591795e-04],
          [-5.936526e-04,  4.759874e-04, -2.091079e-04, -3.034194e-04]]]],
       device='cuda:0')
---
4 torch.Size([1, 2, 4, 4]) torch.Size([1, 2, 4, 4]) torch.Size([1, 2, 4, 2, 3, 2, 4]) torch.Size([1, 2, 4, 4, 4])
tensor([[[[-1.228441e-04,  1.492486e-04, -4.761901e-04, -1.169280e-04],
          [-5.437012e-04,  2.777719e-04,  1.680864e-04, -7.518199

In [9]:
# TODO XXX: think what to do about the device here?
BIG_NR=1_000_000
def _p_gen_aux_split(device, p_gen_aux=None, n=2):
    if p_gen_aux is None:
        return [None] * n
    generator = torch.Generator(device=device).manual_seed(p_gen_aux)
    return torch.randint(0, BIG_NR, (n,), device=device, generator=generator).tolist()
device="cuda"
aa, bb = _p_gen_aux_split(device, 42, 2)

In [3]:
# t_gpt2_tlayer_fwd_sublock2_fwd Train=False
import torch
#torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 1, 2, 2
layer_params = init_tlayer_gpt2(D, H, 4*D, 1) #, 0.00000001)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
y = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, False)
res = t_gpt2_tlayer_sublock2_fwd(layer_params[-6:], y, False)
print(res.shape)
print(res-y)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock2_fwd, train=False)
res = jacrev(fn)(layer_params[-6:], y)
print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
print(res[0])
print(res[1])
print(res[-2])
print(res[-1])
print("---")

res2 = t_gpt2_tlayer_sublock2_bkwd_p(layer_params[-6:], y, train=False)
print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
print(res2[0])
print(res2[1])
print(res2[-2])
print(res2[-1])
print("---")

# TODO XXX: The below one is tricky to test I think
# res = jacrev(fn, argnums=1)(layer_params[-6:], y)
# print(res.shape, res)
# print("---")

# from model_triton import _mult_jacs_in_2d

# res2 = t_gpt2_tlayer_sublock2_bkwd_x(layer_params[-6:], y, train=False)
# print(res2.shape, res2)

torch.Size([1, 2, 2])
tensor([[[-0.0004, -0.0004],
         [-0.0004, -0.0004]]], device='cuda:0')
---
6 torch.Size([1, 2, 2, 2]) torch.Size([1, 2, 2, 2]) torch.Size([1, 2, 2, 8, 2]) torch.Size([1, 2, 2, 8])
tensor([[[[-3.2364e-04, -4.2486e-05],
          [-4.6472e-04,  7.8919e-05]],

         [[-3.2364e-04, -4.2486e-05],
          [-4.6472e-04,  7.8919e-05]]]], device='cuda:0')
tensor([[[[ 4.5769e-04, -6.0085e-05],
          [ 6.5722e-04,  1.1161e-04]],

         [[ 4.5769e-04, -6.0085e-05],
          [ 6.5722e-04,  1.1161e-04]]]], device='cuda:0')
tensor([[[[[ 0.0055,  0.0161, -0.0165,  0.0053,  0.0069, -0.0065,  0.0049,
             0.0076],
           [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
             0.0000]],

          [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
             0.0000],
           [ 0.0055,  0.0161, -0.0165,  0.0053,  0.0069, -0.0065,  0.0049,
             0.0076]]],


         [[[ 0.0055,  0.0161, -0.0165,  0.0053,  0

In [1]:
# t_gpt2_tlayer_fwd_sublock2_fwd Train=True
import torch
#torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 1, 2, 2
p_gen_aux = 42
layer_params = init_tlayer_gpt2(D, H, 4*D, 1) #, 0.00000001)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
y = t_gpt2_tlayer_sublock1_fwd(layer_params[:-6], y, mask, False) # this can be train=False
res = t_gpt2_tlayer_sublock2_fwd(layer_params[-6:], y, True, p_gen_aux)
print(res.shape)
print(res-y)
print("---")

from functools import partial
fn = partial(t_gpt2_tlayer_sublock2_fwd, train=True, p_gen_aux=p_gen_aux)
# res = jacrev(fn)(layer_params[-6:], y)
# print(len(res), res[0].shape, res[1].shape, res[2].shape, res[3].shape)
# print(res[0])
# print(res[1])
# print(res[-2])
# print(res[-1])
# print("---")

# res2 = t_gpt2_tlayer_sublock2_bkwd_p(layer_params[-6:], y, train=True, p_gen_aux=p_gen_aux)
# print(len(res2), res2[0].shape, res2[1].shape, res2[2].shape, res2[3].shape)
# print(res2[0])
# print(res2[1])
# print(res2[-2])
# print(res2[-1])
# print("---")

#TODO XXX: The below one is tricky to test I think
res = jacrev(fn, argnums=1)(layer_params[-6:], y)
print(res.shape, res)
print("---")

from model_triton import _mult_jacs_in_2d

res2 = t_gpt2_tlayer_sublock2_bkwd_x(layer_params[-6:], y, train=True, p_gen_aux=p_gen_aux)
print(res2.shape, res2)

torch.Size([1, 2, 2])
tensor([[[ 0.6491, -1.0533],
         [-0.9334, -2.6817]]], device='cuda:0')
---
torch.Size([1, 2, 2, 1, 2, 2]) tensor([[[[[[ 1.4552e-11,  0.0000e+00],
            [ 0.0000e+00,  0.0000e+00]]],


          [[[-1.0914e-11,  1.0914e-11],
            [ 0.0000e+00,  0.0000e+00]]]],



         [[[[ 0.0000e+00,  0.0000e+00],
            [-1.4552e-11,  5.8208e-11]]],


          [[[ 0.0000e+00,  0.0000e+00],
            [ 7.2760e-12, -2.1828e-11]]]]]], device='cuda:0')
---
torch.Size([1, 2, 2, 1, 2, 2]) tensor([[[[[[ 3.5061e-11, -3.5061e-11],
            [ 0.0000e+00,  0.0000e+00]]],


          [[[-7.6507e-12,  7.6507e-12],
            [ 0.0000e+00,  0.0000e+00]]]],



         [[[[ 0.0000e+00,  0.0000e+00],
            [-2.6688e-11,  5.9933e-11]]],


          [[[ 0.0000e+00,  0.0000e+00],
            [ 5.0634e-12, -1.2318e-11]]]]]], device='cuda:0')


In [28]:
# t_tlayer_fwd_gpt2 + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 2, 2, 4
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_fwd(layer_params, y, mask, False)
print(res.shape)
print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts


from functools import partial
fn = partial(t_gpt2_tlayer_fwd, mask=mask, train=False)
res = jacrev(fn)(layer_params, y)
print_res_shapes(res)
print("---XXX---")

res2 = t_gpt2_tlayer_bkwd_p(layer_params, y, mask, train=False)
print_res_shapes(res2)
print("---XXX---")

# res = jacrev(fn, argnums=1)(layer_params, y)
# print(res.shape)
# print(res)
# print("---")

# res2 = t_gpt2_tlayer_bkwd_x(layer_params, y, mask, train=False)
# print(res2.shape)
# print(res2)


torch.Size([2, 2, 4])
---
10
- 2
-- torch.Size([2, 4, 4]) tensor([-5.724964e-05,  9.969356e-04,  1.239672e-04], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([ 2.495298e-04, -2.664233e-04, -9.784271e-05], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tensor([-0.000383, -0.000923, -0.000587], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([-0.000382, -0.000922, -0.000586], device='cuda:0')
- 2
-- torch.Size([2, 4, 2, 3, 2, 4]) tensor([-0.002566,  0.018484,  0.003612], device='cuda:0')
-- torch.Size([2, 4, 2, 3, 2, 4]) tensor([ 0.011171, -0.004898, -0.002812], device='cuda:0')
- 2
-- torch.Size([2, 4, 4, 4]) tensor([ 0.001943,  0.031369, -0.006014], device='cuda:0')
-- torch.Size([2, 4, 4, 4]) tensor([ 0.004878, -0.011782, -0.013972], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tensor([ 1.968648e-04,  3.995623e-04, -4.111990e-05], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([ 0.000222, -0.000427, -0.000456], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tensor([-0.000453, -0.

In [2]:
# t_tlayer_fwd_gpt2 + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 2, 2, 4
p_gen_aux = [42, 43, 44]
layer_params = init_tlayer_gpt2(D, H, 4*D, 1)
y= torch.randn((BS, N, D), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayer_fwd(layer_params, y, mask, True, p_gen_aux=p_gen_aux)
print(res.shape)
print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts


from functools import partial
fn = partial(t_gpt2_tlayer_fwd, mask=mask, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layer_params, y)
print_res_shapes(res)
print("---XXX---")

res2 = t_gpt2_tlayer_bkwd_p(layer_params, y, mask, train=True, p_gen_aux=p_gen_aux)
print_res_shapes(res2)
print("---XXX---")

# res = jacrev(fn, argnums=1)(layer_params, y)
# print(res.shape)
# print(res)
# print("---")

# res2 = t_gpt2_tlayer_bkwd_x(layer_params, y, mask, train=False)
# print(res2.shape)
# print(res2)


torch.Size([2, 2, 4])
---
10
- 2
-- torch.Size([2, 4, 4]) tensor([2.081779e-04, 4.265390e-04, 1.291633e-06], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([1.358328e-05, 2.727477e-04, 3.581074e-06], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tensor([-2.026328e-04,  6.400786e-04, -8.274857e-06], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([-2.031429e-04,  6.401073e-04, -7.973516e-06], device='cuda:0')
- 2
-- torch.Size([2, 4, 2, 3, 2, 4]) tensor([-0.006227,  0.004037, -0.001009], device='cuda:0')
-- torch.Size([2, 4, 2, 3, 2, 4]) tensor([-0.000403,  0.002582, -0.002552], device='cuda:0')
- 2
-- torch.Size([2, 4, 4, 4]) tensor([-0.017456, -0.039551,  0.000631], device='cuda:0')
-- torch.Size([2, 4, 4, 4]) tensor([ 0.001274, -0.016754, -0.002354], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tensor([ 2.511118e-04, -1.108891e-03, -6.672889e-05], device='cuda:0')
-- torch.Size([2, 4, 4]) tensor([-0.000299, -0.000188,  0.000237], device='cuda:0')
- 2
-- torch.Size([2, 4, 4]) tens

In [41]:
y = torch.randn((2, N))
torch.arange(y.shape[1]).unsqueeze(0).expand(*y.shape)
#.expand(y.shape[0], 1)
#res

tensor([[0, 1],
        [0, 1]])

In [3]:
import torch
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *

BS, H, N, D = 1, 1, 2, 2
vocab_size=12
layers=1
params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
def fn(params, y):
    return y + params
y = t_embed_fwd(params[0], y)
print(y.shape)
res= fn(params[1][0], y)
print(res.shape)

res = jacrev(fn)(params[1][0], y)
print(res)

def fn_bkwd(params, y):
    y = y.reshape((-1, y.shape[-1]))
    indices = torch.arange(y.shape[1], device=y.device).unsqueeze(0).expand(*y.shape) # we 
    print(indices)
    jac_pos_enc = list(t_embed_bkwd(params[1], indices))
    print(f'len', len(jac_pos_enc))
    #jac_pos_enc[0]
    jac_pos_enc[0][jac_pos_enc[0]!=0] = 1
#    jac_pos_enc = [jac_pos_enc[0], ]
#    jac_pos_enc = [jac_pos_enc[0] / (params[1][0].shape[1] * 2), ]
    return jac_pos_enc

res = fn_bkwd(params, y)
print(res[0])

torch.Size([1, 2, 2])
torch.Size([1, 2, 2])
tensor([[[[[1., 0.],
           [0., 0.]],

          [[0., 1.],
           [0., 0.]]],


         [[[0., 0.],
           [1., 0.]],

          [[0., 0.],
           [0., 1.]]]]], device='cuda:0')
tensor([[0, 1],
        [0, 1]], device='cuda:0')
len 1
tensor([[[[[1., 0.],
           [0., 0.]],

          [[0., 1.],
           [0., 0.]]],


         [[[0., 0.],
           [1., 0.]],

          [[0., 0.],
           [0., 1.]]]],



        [[[[1., 0.],
           [0., 0.]],

          [[0., 1.],
           [0., 0.]]],


         [[[0., 0.],
           [1., 0.]],

          [[0., 0.],
           [0., 1.]]]]], device='cuda:0')


In [6]:
# t_gpt2_tlayers_fwd + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
vocab_size = 6
layers = 2
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayers_fwd(layers_params, y, mask, None, False)
print(res.shape)

print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_gpt2_tlayers_fwd, mask=mask, indices=None, train=False)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")

res2 = t_gpt2_tlayers_bkwd_p(layers_params, y, mask, None, train=False)
print_res_shapes(res2) 
#print(res2[0])
#print(res2[1])

torch.Size([1, 2, 4])
---
5
- 2
-- torch.Size([1, 2, 4, 6, 4]) tensor([0., 0., 0.], device='cuda:0')
-- torch.Size([1, 2, 4, 6]) tensor([0., 0., 0.], device='cuda:0')
- 1
-- torch.Size([1, 2, 4, 2, 4]) tensor([-4.178149, -3.614153,  9.409573], device='cuda:0')
- 10
-- torch.Size([1, 2, 4, 4]) tensor([ 0.000436, -0.003398,  0.001194], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([-0.000888,  0.005572, -0.003817], device='cuda:0')
-- torch.Size([1, 2, 4, 2, 3, 2, 4]) tensor([-0.070617, -0.087650, -0.044986], device='cuda:0')
-- torch.Size([1, 2, 4, 4, 4]) tensor([-0.201005, -0.245705,  0.369999], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([-5.039283e-03,  6.397760e-05, -1.143041e-03], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([ 0.005527, -0.000223,  0.004835], device='cuda:0')
-- torch.Size([1, 2, 4, 16, 4]) tensor([0.042090, 0.012909, 0.010935], device='cuda:0')
-- torch.Size([1, 2, 4, 16]) tensor([ 0.088869, -0.090330, -0.046223], device='cuda:0')
-- torch.Siz

In [7]:
# t_gpt2_tlayers_fwd + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 1, 2, 2, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_tlayers_fwd(layers_params, y, mask, None, True, p_gen_aux=p_gen_aux)
print(res.shape)

print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_gpt2_tlayers_fwd, mask=mask, indices=None, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")

res2 = t_gpt2_tlayers_bkwd_p(layers_params, y, mask, None, True, p_gen_aux)
print_res_shapes(res2) 
#print(res2[0])
#print(res2[1])

torch.Size([1, 2, 4])
---
5
- 2
-- torch.Size([1, 2, 4, 6, 4]) tensor([0., 0., 0.], device='cuda:0')
-- torch.Size([1, 2, 4, 6]) tensor([0., 0., 0.], device='cuda:0')
- 1
-- torch.Size([1, 2, 4, 2, 4]) tensor([-4.556520, -1.069513, 13.938005], device='cuda:0')
- 10
-- torch.Size([1, 2, 4, 4]) tensor([-0.003073, -0.003849,  0.001485], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([-0.013222,  0.004133, -0.004684], device='cuda:0')
-- torch.Size([1, 2, 4, 2, 3, 2, 4]) tensor([ 0.034813, -0.139275, -0.047691], device='cuda:0')
-- torch.Size([1, 2, 4, 4, 4]) tensor([-0.135330, -0.044505,  0.306945], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([-0.001051, -0.015113,  0.003353], device='cuda:0')
-- torch.Size([1, 2, 4, 4]) tensor([0.004411, 0.011853, 0.007323], device='cuda:0')
-- torch.Size([1, 2, 4, 16, 4]) tensor([-0.006158, -0.032420,  0.011833], device='cuda:0')
-- torch.Size([1, 2, 4, 16]) tensor([ 0.112122, -0.007073,  0.025366], device='cuda:0')
-- torch.Size([1, 2, 4, 

In [11]:
# t_gpt2_forward + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 2, 2, 4
vocab_size = 6
layers = 2
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_forward(layers_params, y, mask, None, False)

print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_gpt2_forward, y_mask=mask, y_indices=None, train=False)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")
for i, i_mask in enumerate(mask):
    #mask[i] = torch.tril(i_mask)
    mask[i] = torch.zeros_like(i_mask)
print(mask)
fn = partial(t_gpt2_forward, y_mask=mask, y_indices=None, train=False)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")

res2 = t_gpt2_bkwd_p(layers_params, y, mask, None, train=False)
print_res_shapes(res2) 
#print(res2[0])
#print(res2[1])

---
5
- 2
-- torch.Size([2, 2, 6, 6, 4]) tensor([-0.023876,  0.895180,  0.913492], device='cuda:0')
-- torch.Size([2, 2, 6, 6]) tensor([0., 0., 1.], device='cuda:0')
- 1
-- torch.Size([2, 2, 6, 2, 4]) tensor([-0.027715,  0.554218, -0.194621], device='cuda:0')
- 10
-- torch.Size([2, 2, 6, 4]) tensor([-1.625612e-05, -4.947522e-05,  2.133402e-05], device='cuda:0')
-- torch.Size([2, 2, 6, 4]) tensor([-7.278824e-05, -2.352890e-04, -2.047363e-04], device='cuda:0')
-- torch.Size([2, 2, 6, 2, 3, 2, 4]) tensor([ 0.001080,  0.001016, -0.000505], device='cuda:0')
-- torch.Size([2, 2, 6, 4, 4]) tensor([ 0.000274,  0.000275, -0.001408], device='cuda:0')
-- torch.Size([2, 2, 6, 4]) tensor([ 1.603317e-05, -3.487642e-05, -4.740024e-05], device='cuda:0')
-- torch.Size([2, 2, 6, 4]) tensor([ 2.512884e-04,  1.676124e-04, -3.700890e-05], device='cuda:0')
-- torch.Size([2, 2, 6, 16, 4]) tensor([-4.349073e-05,  1.361640e-04, -8.490586e-04], device='cuda:0')
-- torch.Size([2, 2, 6, 16]) tensor([-0.000826,  0

In [2]:
# t_gpt2_forward + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_tlayer_gpt2
from model_triton import *
BS, H, N, D = 2, 2, 5, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_gpt2_forward(layers_params, y, mask, None, True, p_gen_aux)
print(f'res.shape', res.shape)

print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_gpt2_forward, y_mask=mask, y_indices=None, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")
for i, i_mask in enumerate(mask):
    #mask[i] = torch.tril(i_mask)
    mask[i] = torch.zeros_like(i_mask)
print(mask)
fn = partial(t_gpt2_forward, y_mask=mask, y_indices=None, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn)(layers_params, y)
print_res_shapes(res)
print("---XXXX--")

res2 = t_gpt2_bkwd_p(layers_params, y, mask, None, train=True, p_gen_aux=p_gen_aux)
print_res_shapes(res2) 
#print(res2[0])
#print(res2[1])

res.shape torch.Size([2, 5, 6])
---
5
- 2
-- torch.Size([2, 5, 6, 6, 4]) tensor([-0.137782,  1.369160, -0.406006], device='cuda:0')
-- torch.Size([2, 5, 6, 6]) tensor([0., 0., 1.], device='cuda:0')
- 1
-- torch.Size([2, 5, 6, 5, 4]) tensor([-0.142560,  0.030283,  0.000000], device='cuda:0')
- 10
-- torch.Size([2, 5, 6, 4]) tensor([9.807979e-05, 5.954188e-05, 1.136734e-05], device='cuda:0')
-- torch.Size([2, 5, 6, 4]) tensor([ 1.731989e-04,  1.015569e-04, -1.746008e-05], device='cuda:0')
-- torch.Size([2, 5, 6, 2, 3, 2, 4]) tensor([-0.000856, -0.001002,  0.000880], device='cuda:0')
-- torch.Size([2, 5, 6, 4, 4]) tensor([ 0.001300,  0.004768, -0.002701], device='cuda:0')
-- torch.Size([2, 5, 6, 4]) tensor([-1.308097e-06, -2.329485e-06,  4.134895e-05], device='cuda:0')
-- torch.Size([2, 5, 6, 4]) tensor([-9.912897e-06, -1.599405e-06, -9.898079e-05], device='cuda:0')
-- torch.Size([2, 5, 6, 16, 4]) tensor([-1.658287e-06, -2.319309e-05,  8.260810e-06], device='cuda:0')
-- torch.Size([2, 5, 

In [5]:
# t_avg_cross_entropy_loss
import torch
from torch.func import jacrev
from loss_and_optimizer_triton import t_avg_cross_entropy_loss, t_avg_cross_entropy_loss_bkwd
from model_triton import t_log_softmax_fwd, t_log_softmax_bkwd
BS, N, V = 4, 2, 3
y_labels = torch.randint(0, V, (BS, N), device="cuda")
y_logits = torch.randn((BS, N, V), device="cuda")
print("y_labels", y_labels)

output = t_avg_cross_entropy_loss(y_labels, y_logits)
print(len(output), output[0])
print('---')

res = jacrev(t_avg_cross_entropy_loss, argnums=1, has_aux=True)(y_labels, y_logits)
print(len(res), res[0].shape)
print(res[0])
print('--')

res2 = t_avg_cross_entropy_loss_bkwd(y_labels, y_logits)
print(res2.shape)
print(res2)
print('--') 

y_labels tensor([[2, 0],
        [2, 1],
        [2, 0],
        [2, 1]], device='cuda:0')
2 tensor(1.605373, device='cuda:0')
---
2 torch.Size([4, 2, 3])
tensor([[[ 0.021246,  0.117412, -0.138658],
         [ 0.000000,  0.000000,  0.000000]],

        [[ 0.015352,  0.091664, -0.107017],
         [ 0.060215, -0.130583,  0.070369]],

        [[ 0.103066,  0.052031, -0.155097],
         [ 0.000000,  0.000000,  0.000000]],

        [[ 0.093773,  0.005562, -0.099334],
         [ 0.011291, -0.136737,  0.125446]]], device='cuda:0')
--
torch.Size([4, 2, 3])
tensor([[[ 0.021246,  0.117412, -0.138658],
         [ 0.000000,  0.000000,  0.000000]],

        [[ 0.015352,  0.091664, -0.107017],
         [ 0.060215, -0.130583,  0.070369]],

        [[ 0.103066,  0.052031, -0.155097],
         [ 0.000000,  0.000000,  0.000000]],

        [[ 0.093773,  0.005562, -0.099334],
         [ 0.011291, -0.136737,  0.125446]]], device='cuda:0')
--


In [2]:
# t_loss + train=False
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd
BS, H, N, D = 2, 2, 2, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_loss(layers_params, y, mask, None, False)
print(res)
print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_loss, y_mask=mask, y_indices=None, train=False)
res = jacrev(fn, has_aux=True)(layers_params, y)
print(res[1])
print_res_shapes(res[0])
print("---XXXX--")
# for i, i_mask in enumerate(mask):
#     mask[i] = torch.tril(i_mask)
#     #mask[i] = torch.zeros_like(i_mask)
# print(mask)
# fn = partial(t_loss, y_mask=mask, y_indices=None, train=False)
# res = jacrev(fn, has_aux=True)(layers_params, y)
# #print_res_shapes(res[0])
# print("---XXXX--")

res2 = t_loss_bkwd(layers_params, y, mask, None, train=False)
print(res2[1])
print_res_shapes(res2[0]) 
#print(res2[0])
#print(res2[1])

(tensor(1.794387, device='cuda:0'), (tensor(1.794387, device='cuda:0'), tensor(0., device='cuda:0'), tensor(1., device='cuda:0')))
---
(tensor(1.794387, device='cuda:0'), tensor(0., device='cuda:0'), tensor(1., device='cuda:0'))
5
- 2
-- torch.Size([6, 4]) tensor([-0.127124,  0.055105,  0.100168], device='cuda:0')
-- torch.Size([6]) tensor([ 0.164353,  0.163971, -0.580547], device='cuda:0')
- 1
-- torch.Size([2, 4]) tensor([-0.628850, -0.075747,  0.011350], device='cuda:0')
- 10
-- torch.Size([4]) tensor([ 3.068704e-05,  1.720320e-04, -1.846318e-04], device='cuda:0')
-- torch.Size([4]) tensor([ 0.000125, -0.000561, -0.000137], device='cuda:0')
-- torch.Size([2, 3, 2, 4]) tensor([-0.006074,  0.003219, -0.009708], device='cuda:0')
-- torch.Size([4, 4]) tensor([-0.001385,  0.004992, -0.003819], device='cuda:0')
-- torch.Size([4]) tensor([ 1.371253e-04, -2.176035e-05, -4.429675e-04], device='cuda:0')
-- torch.Size([4]) tensor([-4.576159e-04,  1.020413e-05, -4.484524e-04], device='cuda:0')


In [1]:
# t_loss + train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd
BS, H, N, D = 2, 2, 5, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")
res = t_loss(layers_params, y, mask, None, True, p_gen_aux)
print(res)
print("---")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

from functools import partial
fn = partial(t_loss, y_mask=mask, y_indices=None, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn, has_aux=True)(layers_params, y)
print(res[1])
print_res_shapes(res[0])
print("---XXXX--")
for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
print(mask)
fn = partial(t_loss, y_mask=mask, y_indices=None, train=True, p_gen_aux=p_gen_aux)
res = jacrev(fn, has_aux=True)(layers_params, y)
print_res_shapes(res[0])
print("---XXXX--")

res2 = t_loss_bkwd(layers_params, y, mask, None, True, p_gen_aux)
print(res2[1])
print_res_shapes(res2[0]) 
#print(res2[0])
#print(res2[1])

(tensor(1.790465, device='cuda:0'), (tensor(1.790465, device='cuda:0'), tensor(0.222222, device='cuda:0'), tensor(0.900000, device='cuda:0')))
---
(tensor(1.790465, device='cuda:0'), tensor(0.222222, device='cuda:0'), tensor(0.900000, device='cuda:0'))
5
- 2
-- torch.Size([6, 4]) tensor([-0.045141, -0.095859,  0.105749], device='cuda:0')
-- torch.Size([6]) tensor([ 0.054522, -0.051900, -0.053196], device='cuda:0')
- 1
-- torch.Size([5, 4]) tensor([-0.287275, -0.061483,  0.133037], device='cuda:0')
- 10
-- torch.Size([4]) tensor([ 2.621366e-05,  1.031182e-05, -8.969118e-05], device='cuda:0')
-- torch.Size([4]) tensor([-7.089571e-05,  1.040890e-05,  3.286575e-04], device='cuda:0')
-- torch.Size([2, 3, 2, 4]) tensor([ 0.002025, -0.004901,  0.003491], device='cuda:0')
-- torch.Size([4, 4]) tensor([ 0.002544, -0.001320, -0.002485], device='cuda:0')
-- torch.Size([4]) tensor([ 1.329654e-04, -2.255055e-04,  1.822136e-05], device='cuda:0')
-- torch.Size([4]) tensor([-6.698675e-04, -4.004006e-0

In [3]:
# Derisk jac_p idea.
x = torch.zeros((2,3))
generator = torch.Generator(device=x.device)
generator.manual_seed(42)
torch.bernoulli(torch.full_like(x, 0.9), generator=generator)

tensor([[1., 0., 1.],
        [0., 1., 1.]])

In [4]:
# Back to basics
q, k, v = qkv.unbind(dim=2)
q = q.reshape((-1,q.shape[-1]))
k = k.reshape((-1,k.shape[-1]))
v = v.reshape((-1, v.shape[-1]))
print(q, k, v)

def qk_fwd(q, k):
    return torch.matmul(q, k.transpose(-2, -1))

def qkv_fwd(q, k, v):
    return torch.matmul(qk_fwd(q, k), v)

print(qkv_fwd(q, k, v).shape)
print("---")
#res = jacrev(qk_fwd)(q, k)
res = jacrev(qkv_fwd)(q, k, v)
print(res.shape)
print(res)
print("---")

NameError: name 'qkv' is not defined

In [121]:
def qk_bkwd(q, k):
    return k.transpose
def qkv_bkwd(q, k, v):
    d_qk_wd_dq = jacrev(qk_fwd)(q, k)

    res = torch.matmul(v.transpose(1, 0), d_qk_wd_dq.transpose(1, 0).reshape(q.shape[0], -1))
    res = res.reshape(res.shape[0], -1, q.numel())
    res = res.transpose(1, 0)
    res = res.reshape(q.shape + q.shape)
    
    return res

#res2 = qk_bkwd(q, k)
res2 = qkv_bkwd(q, k, v)
print(res2.shape)
print(res2)

torch.Size([2, 3, 2, 3])
tensor([[[[-0.033605, -2.002479, -1.487947],
          [ 0.000000,  0.000000,  0.000000]],

         [[-0.382970, -1.029678,  0.075037],
          [ 0.000000,  0.000000,  0.000000]],

         [[ 0.180972, -0.494185, -0.802026],
          [ 0.000000,  0.000000,  0.000000]]],


        [[[ 0.000000,  0.000000,  0.000000],
          [-0.033605, -2.002479, -1.487947]],

         [[ 0.000000,  0.000000,  0.000000],
          [-0.382970, -1.029678,  0.075037]],

         [[ 0.000000,  0.000000,  0.000000],
          [ 0.180972, -0.494185, -0.802026]]]], device='cuda:0')


In [39]:
torch.block_diag(torch.ones(2), torch.ones(2))

tensor([[1., 1., 0., 0.],
        [0., 0., 1., 1.]])

In [38]:
def test_fn(a):
    return torch.sum(a, dim=1)

aa = torch.randn((2,2))
print(aa)
print(test_fn(aa))
res = jacrev(test_fn)(aa)
print(res)
print(res.shape)

tensor([[-0.8190, -1.8277],
        [-0.7926,  0.6985]])
tensor([-2.6466, -0.0941])
tensor([[[1., 1.],
         [0., 0.]],

        [[0., 0.],
         [1., 1.]]])
torch.Size([2, 2, 2])


In [24]:
def t_test_fwd(x):
    return 0.044715 * torch.pow(x,3)

def t_test_bkwd(x):
    return 0.044715 * 3 * torch.pow(x,2)

def t_test_fwd(x):
    k = math.sqrt(2/math.pi)
    tanh_term = torch.tanh(k * (x + 0.044715 * torch.pow(x,3)))
    return tanh_term

def t_test_bkwd(x):
    k = math.sqrt(2/math.pi)
    tanh_term = torch.tanh(k * (x + 0.044715 * torch.pow(x,3)))
    tanh_dx = (1 - torch.pow(tanh_term, 2)) * k * ( 1 + 3 * 0.044715 * torch.pow(x,2))
    return tanh_dx


# print(vjp(t_test_fwd, aa)[0])
# print(t_test_bkwd(aa) * aa)

# test_aa = torch.tensor(1.1)
# print(grad(t_test_fwd)(test_aa))
# print(t_test_bkwd(test_aa))

print(jacrev(t_test_fwd)(aa))
print(t_test_bkwd(aa))


tensor([[[[0.7855, 0.0000],
          [0.0000, 0.0000]],

         [[0.0000, 0.5382],
          [0.0000, 0.0000]]],


        [[[0.0000, 0.0000],
          [0.7643, 0.0000]],

         [[0.0000, 0.0000],
          [0.0000, 0.7812]]]])
tensor([[0.7855, 0.5382],
        [0.7643, 0.7812]])
